# Yapay sinir ağları sınıflandırma

Hocam zaten diper file 'da da teoric partı göreceğin üzere zaten bir sinir ağı yapısnı biiyorsun. 

Önce gözlemelerdeki bağımsız değişkenler ağırlıklarla(yani katsayılarla) çarpılıyor 

daha sonra bunlar toplanıyuor üzrine birde bir aktivasyon fonksiyonu elde ediliyor son olarak tahmin değeri elde edilip gerçek 

değerle karşilaştırılıyor. işte geriye doğru öğrenme kımsı tam olarak  burada kendini gösteriyor. 

ilk başta o çarptığımız katsayıalrı ağırlıkları random atar sonra tekrar tekrar deneyerk işte bu çıktıları karşılaştırarak doğtru katsayıyı bulur

Ez cümle önce topolojik yapı(katman saysı falan) ayarlanır sonra random katsayılala işlem başlar daha sonra çıktılara göre 

en iyi paremetreler ağırlıklar bulunmaya çalışılır.

In [6]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
diabet=pd.read_csv("diabetes.csv")
df=diabet.copy()
df=df.dropna()
y=df["Outcome"]
X=df.drop(["Outcome"],axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.30, random_state=42) 

In [11]:
# öncelikle bağımsız değişkenleri standardlaştıralım. Normalde algoritmalrın hepsi standartlaştırlmış veriyi sever ancak
# yapay sşnşr ağları özellikle bu konuda hassatır aykırı veri, değişkenleirn varyansı bunlardan oldukça fazla etkilenir
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [10]:
from sklearn.neural_network import MLPClassifier #mlpc multi layer perceptron classifier

In [12]:
mlpc=MLPClassifier()
mlpc.fit(X_train_scaled,y_train)
mlpc.score(X_test_scaled,y_test)

0.7359307359307359

# Tuning

In [13]:
mlpc.hidden_layer_sizes
#bu tek katlı olduğunu gösterir. tek katlı ve 100 hücresi var.

(100,)

In [14]:
mlpc.activation#aktivasyon fonksiyonu olarak relu kullanılmış

'relu'

In [15]:
mlpc.solver#baştaki ağırllıların optimumm değerini bulmak için kullanılan metod

'adam'

In [16]:
mlpc.alpha#düzenlem terimi

0.0001

In [21]:
?mlpc
#gördüğün üzere pek çok paremetre var.  biz burada daha giriş levelinde olduğumuz için en çok etkili 
# sadece bir kaç paremetre üzerinde çalışıcaz ama gerçekte bir veri üzerinde çalışırken başka paremetrelerle de oynanabilir

Type:        MLPClassifier
String form: MLPClassifier()
File:        ~/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py
Docstring:  
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'identity', no-op activation, useful to implement linear bottleneck,
      returns f(x) = x

    - 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    - 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    - 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)

solver : {'lbfgs', 'sgd', 'adam'}, d

# Tuning

In [22]:
params={"alpha":[0.1,0.01,0.02,0.005, 0.0001,0.00001],
        "hidden_layer_sizes":[(10,10,10),(100,100,100),(100,100),(3,5),(5,3)],
        "solver":["lbfgs","adam","sgd"],
        "activation":["relu","logistic"]}

In [23]:
mlpc=MLPClassifier()
mlpc_cv=GridSearchCV(mlpc, params, cv=10, n_jobs=-1, verbose=2)
mlpc_cv.fit(X_train_scaled,y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarni

[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s


/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarni

[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=lbfgs; total time=   0.1s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=adam; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=adam; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=adam; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=adam; total time=   0.2s
[CV] END activation=relu, alpha=0.1, hidden_layer_sizes=(10, 10, 10), solver=adam; total time=   0.2s


...

/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarni

[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.1s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.2s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.1s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.2s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.2s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.2s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=sgd; total time=   0.0s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=adam; total time=   0.2s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=sgd; total time=   0.1s
[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=sgd; to

/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/necmiyesoyla/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  war

[CV] END activation=logistic, alpha=1e-05, hidden_layer_sizes=(5, 3), solver=sgd; total time=   0.1s


GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005, 0.0001, 1e-05],
                         'hidden_layer_sizes': [(10, 10, 10), (100, 100, 100),
                                                (100, 100), (3, 5), (5, 3)],
                         'solver': ['lbfgs', 'adam', 'sgd']},
             verbose=2)

In [24]:
mlpc_cv.best_params_

{'activation': 'logistic',
 'alpha': 0.005,
 'hidden_layer_sizes': (100, 100),
 'solver': 'adam'}

In [28]:
mlpc_tuned=MLPClassifier(activation="logistic",alpha=0.005,hidden_layer_sizes=(100, 100),solver="adam")
mlpc_tuned.fit(X_train_scaled,y_train)
mlpc_tuned.score(X_test_scaled,y_test)

0.7359307359307359